In [1]:
import pandas as pd
import numpy as np

In [73]:
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:

btdf = pd.read_parquet('processed_big.parquet')
cit_df = pd.read_parquet('big_ai_dataset.parquet')
cdf = cit_df[['paper_id','inbound_citations','outbound_citations']]
df = pd.merge(btdf,cdf,on='paper_id')

In [3]:
df = pd.read_parquet('meme_score.parquet')

sum inbound_citations that has meme of publication with this meme and\sum inbound_citations of publication with this meme
sum (pub that has a meme and does not have a meme-carrer in inbound_citations)/sum(pub that do not cite meme carrers)

In [4]:
meme_list = dict()
memes=[]
for chunks_list in df['noun_chunks_cleaned']:
    for chunk in chunks_list:
        if chunk not in meme_list:
            meme_list[chunk]=1
            memes.append(chunk)
        else:
            meme_list[chunk]+=1

In [5]:
df.head()

,paper_id,noun_chunks_cleaned,inbound_citations,outbound_citations,outbound_citations_cleared,inbound_citations_clear,outbound_citations_clear
0,199668001,"[road construction project, territory, republi...",[],"[106476531, 67083539, 36731616, 34616216, 1096...",NaN,[],[]
1,2879234,"[convolutional neural net, cnn, remarkable per...","[11015941, 52160763, 199488257, 23874112, 4241...","[206592419, 215721, 10111903, 1003907, 3198903...",NaN,"[340420, 471907, 250792]","[215721, 127386, 392527]"
2,17786914,"[excellent result, benchmark, acoustic modelin...",[198931159],"[9530137, 398770, 207168299, 14832074, 1799800...",NaN,[],"[398770, 299222]"
3,17432300,"[east asian character, rich hierarchical struc...","[15983137, 7625356, 15404413, 199472639, 47640...","[371064, 3246932, 8991475, 36725681, 32031694,...",NaN,[467570],[371064]
4,204957502,"[internet, countless article, multimedia conte...",[],[],NaN,[],[]


In [6]:
#for i in df.iterrows:
#    memes[0] in i['noun_chunks_cleaned']#df[]all() not in df['outbound_citations']


In [7]:
inc=[]
out=[]
for i,row in df.iterrows():
    cit =[]
    for cited in row['outbound_citations']:
        cited = int(cited)
        if cited in df['paper_id']:
            cit.append(cited)
        
    out.append(cit)
    cit=[]
    for citing in row['inbound_citations']:
        citing = int(citing)
        if citing in df['paper_id']:
            cit.append(citing)
    inc.append(cit)

In [8]:
df['inbound_citations_clear']=inc
df['outbound_citations_clear']=out

In [9]:
df.to_parquet('meme_score.parquet')

In [10]:
df['noun_chunks_cleaned'] = df['noun_chunks_cleaned'].apply(list)

In [13]:
df['noun_chunks_cleaned'].str.contains('benchmark',regex=False)

0         False
1         False
2          True
3         False
4         False
          ...  
504995    False
504996    False
504997    False
504998    False
504999    False
Name: noun_chunks_cleaned, Length: 505000, dtype: bool

In [34]:
len(memes)

3883767

In [61]:
df.index=df.paper_id

In [67]:
df['outbound_citations_clear'].apply(lambda keys:[meme for key in keys for meme in memes[key]])

KeyError: 215721

In [ ]:
for key in keys:
    for meme in memes[key]:
        

In [75]:
enc = MultiLabelBinarizer(sparse_output=True)
memes_enc = enc.fit_transform(df['noun_chunks_cleaned'])

Zrobić wykres częstości memów

In [83]:
memes_enc

<505000x3883767 sparse matrix of type '<class 'numpy.int32'>'
	with 17570452 stored elements in Compressed Sparse Row format>

In [85]:
import seaborn

ModuleNotFoundError: No module named 'seaborn'

In [84]:
memes_enc.sum(axis=0)

matrix([[1442,    1,    1, ...,    1,    1,    1]], dtype=int32)

In [63]:
memes = df['noun_chunks_cleaned']

In [55]:
df['noun_chunks_cleaned'].apply(lambda memes: [])

0         [road construction project, territory, republi...
1         [convolutional neural net, cnn, remarkable per...
2         [excellent result, benchmark, acoustic modelin...
3         [east asian character, rich hierarchical struc...
4         [internet, countless article, multimedia conte...
                                ...                        
504995    [abstract probabilistic model, strength, many ...
504996    [alzheimer disease ad, different course, patie...
504997    [concurrency bug, software testing, non determ...
504998    [new statistic deformable model, discriminant ...
504999    [backgroundthe conservation, sequence, relate ...
Name: noun_chunks_cleaned, Length: 505000, dtype: object

In [51]:
from tqdm import tqdm


p_spark1 = []
p_spark2 = []
p_stick1 = []
p_stick2 = []
for key in tqdm(memes):
    #does it have meme?
    c = df['noun_chunks_cleaned'].str.contains(key,regex=False)
    #does it cite meme carrers?
    c_spark = df['outbound_citations_clear'].str.contains(any(df[c]['paper_id']),regex=False)
    c_stick = ~c_spark
    

    p_spark1.append((c_spark&c).sum())
    p_spark2.append(c_spark.sum())
    p_stick1.append((c_stick&c).sum())
    p_stick2.append(c_stick.sum())

    
    

  0%|          | 5/3883767 [00:11<2498:41:11,  2.32s/it]


KeyboardInterrupt: 

In [52]:
p_spark1

[0, 0, 0, 0, 0]